In [3]:
!pip install scipy

You should consider upgrading via the 'c:\users\mike\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [4]:
import numpy as np
import scipy.optimize as op
# Define the expected returns and covariances of the assets in the portfolio
expected_returns = np.array([0.1, 0.2, 0.15])
covariances = np.array([
    [0.005, 0.001, 0.0005],
    [0.001, 0.004, 0.002],
    [0.0005, 0.002, 0.003]
])

# Set the portfolio constraints
total_budget = 1.0
max_risk = 0.1

# Define the optimization function
def optimize_portfolio(expected_returns, covariances, total_budget, max_risk):
    num_assets = len(expected_returns)

    # Create the weight matrix (random initial weights)
    weights = np.random.random(num_assets)
    weights /= np.sum(weights)

    # Define the optimization problem
    def objective_function(weights):
        # Calculate the portfolio risk
        portfolio_risk = np.sqrt(np.dot(weights.T, np.dot(covariances, weights)))

        # Calculate the portfolio return
        portfolio_return = np.dot(weights.T, expected_returns)

        # Calculate the objective function value (risk-adjusted return)
        obj_value = portfolio_return - max_risk * portfolio_risk

        return obj_value

    # Minimize the objective function using the scipy.optimize library
    bounds = [(0.0, total_budget)] * num_assets
    constraints = {'type': 'eq', 'fun': lambda x: np.sum(x) - total_budget}
    result = op.minimize(
        objective_function,
        weights,
        constraints=constraints,
        bounds=bounds
    )

    return result

# Optimize the portfolio
result = optimize_portfolio(expected_returns, covariances, total_budget, max_risk)

# Print the optimized portfolio weights
print(result.x)


[1.00000000e+00 5.68989300e-16 1.66099773e-16]


In [6]:
import yfinance as yf

# Define a function to calculate the sortino ratio for a given stock
def sortino_ratio(ticker):
    # Get the stock data using yfinance
    data = yf.Ticker(ticker).history(period="max")

    # Calculate the returns
    returns = data["Close"].pct_change().dropna()

    # Calculate the downside deviation
    downside_dev = returns[returns < 0].std()

    # Calculate the sortino ratio
    sortino = returns.mean() / downside_dev
    
    print(ticker, " has sortino of ", sortino)

    return sortino

# Define a function to get the tickers with the highest sortino ratio
def get_top_sortino(tickers):
    # Initialize a dictionary to store the sortino ratio for each ticker
    sortino_dict = {}

    # Loop through the tickers and calculate the sortino ratio for each one
    for ticker in tickers:
        sortino = sortino_ratio(ticker)
        sortino_dict[ticker] = sortino

    # Sort the dictionary by the sortino ratio in descending order
    sorted_dict = {k: v for k, v in sorted(sortino_dict.items(), key=lambda item: item[1], reverse=True)}

    # Return the tickers with the highest sortino ratio
    return list(sorted_dict.keys())[:5]

# Define a list of tickers to test the function
tickers = ["AAPL", "MSFT", "GOOG", "AMZN", "META"]

# Get the tickers with the highest sortino ratio
top_sortino = get_top_sortino(tickers)

# Print the results
print(f"The top tickers with the highest sortino ratio are: {top_sortino}")

AAPL  has sortino of  0.05343886808131467
MSFT  has sortino of  0.07385042123719772
GOOG  has sortino of  0.07213283725528899
AMZN  has sortino of  0.06921316718968024
- FB: No data found, symbol may be delisted
FB  has sortino of  nan
The top tickers with the highest sortino ratio are: ['MSFT', 'GOOG', 'AMZN', 'AAPL', 'FB']


In [46]:
import pandas as pd
from scipy.optimize import minimize, Bounds

returns = pd.DataFrame()
stcks=["MSFT","GOOG","AZO","CVNA","AAPL","MBUU","JNJ"]

for ticker in stcks:    
    data = yf.Ticker(ticker).history(period="max")
    # Calculate the returns
    closes = data["Close"].pct_change().dropna()
    returns = pd.concat([returns,closes],axis=1)

returns = returns.dropna()
returns['ix'] = returns.index
returns = returns.drop(['ix'], axis=1)
# print(returns)

# define the objective function that we want to maximize
def sortino_ratio(weights, returns, target=0):
    # calculate the expected return of the portfolio
    
    portfolio_return = returns.mean() @ weights

    # calculate the downside deviation of the portfolio
    downside_deviation = (returns - target).where(returns < target, 0).std()

    # calculate the Sortino ratio
    sortino_ratio = portfolio_return / downside_deviation

    # return the negative of the Sortino ratio, because we want to maximize it
    # print(sortino_ratio[0])
    return -(sortino_ratio[0])

# define the constraints for the optimization
constraints = [
    {'type': 'eq', 'fun': lambda x: x.sum() - 1},
    {'type': 'ineq', 'fun': lambda x: x}
    # weights must sum to 1
]

# define the initial guess for the portfolio weights
initial_guess = [1/(len(returns.columns))] * (len(returns.columns))

# use the minimize function from the scipy library to find the optimal weights
results = minimize(
    sortino_ratio,
    initial_guess,
    args=(returns, 0),
    constraints=constraints
)

# print the optimized weights
print(results.x)



0.08583684476101817
0.08583684624185467
0.08583684569017365
0.0858368461744971
0.08583684625818638
0.0858368463575753
0.08583684621671861
0.08583684534160213
0.08965887689098022
0.08965887837181671
0.0896588778201357
0.08965887830445912
0.0896588783881484
0.08965887848753734
0.08965887834668064
0.08965887747156417
0.0984940418266569
0.0984940433074934
0.09849404275581237
0.09849404324013583
0.0984940433238251
0.09849404342321402
0.09849404328235732
0.09849404240724086
0.10037239273193405
0.10037239421277054
0.10037239366108953
0.10037239414541298
0.10037239422910224
0.10037239432849117
0.10037239418763447
0.100372393312518
0.10046278403374155
0.10046278551457806
0.10046278496289703
0.10046278544722048
0.10046278553090976
0.10046278563029867
0.10046278548944199
0.1004627846143255
0.1009147405317551
0.1009147420125916
0.10091474146091058
0.10091474194523402
0.1009147420289233
0.10091474212831222
0.10091474198745552
0.10091474111233906
0.10317452320444097
0.10317452468527746
0.10317452413